In [ ]:
import pandas_datareader as pdr
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np
import math
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM


 We will train this LSTM network using Tesla's stock market data

In [ ]:
df=pdr.get_data_tiingo('Stock_symbol',api_key=key)

In [ ]:
df.to_csv('stock.csv')

In [ ]:
data=pd.read_csv('stock.csv')

In [ ]:
data

In [ ]:
# We will consider only the closing price of the stock after 
data1=data.reset_index()["close"]

In [ ]:
data1.shape

(1257,)

In [ ]:
# Getting the data in the range of (0,1) using Min max scaler as LSTMS are sensitive to large input values
scaler=MinMaxScaler(feature_range=(0,1))
data1=scaler.fit_transform(np.array(data1).reshape(-1,1))

In [ ]:
# as u can see the  data has been converted to an array
data1

array([[0.00121372],
       [0.00380138],
       [0.00334016],
       ...,
       [0.43837206],
       [0.46510792],
       [0.46888017]])

In [ ]:
#We will make a 75:25 train test split 
training_size=int(len(data1)*0.75)
test_size=len(data1)-training_size
train_data,test_data=data1[0:training_size,:],data1[training_size:len(data1),:1]

In [ ]:
train_data.shape,test_data.shape

((942, 1), (315, 1))

In [ ]:
#Definig the Function to create training set and test set 
def create_dataset(dataset, time_step):
	data_X, data_Y = [], []
	for i in range(len(dataset)-time_step-1):
		a = dataset[i:(i+time_step), 0]  
		data_X.append(a)
		data_Y.append(dataset[i + time_step, 0])
	return np.array(data_X), np.array(data_Y)

In [ ]:
time_step = 100
X_train, Y_train = create_dataset(train_data, time_step)
X_test, Y_test = create_dataset(test_data, time_step)

In [ ]:
#An LSTM needs 3 inputs 
X_train =X_train.reshape(X_train.shape[0],X_train.shape[1] , 1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1] , 1)

In [ ]:
#Creating a model with 3 LSTM layers 
model=Sequential()
model.add(LSTM(50,return_sequences=True,input_shape=(100,1))) #Layer with 50 activation units
model.add(LSTM(50,return_sequences=True))  #Layer with 50 activation units
model.add(LSTM(65)) #Layer with 65 activation units
model.add(Dense(1))
#Specifying the type of loss and the optimizing Algorithm
model.compile(loss='mean_squared_error',optimizer='adam')

In [ ]:
#Summary of the model 
model.summary()


In [ ]:
model.fit(X_train,Y_train,validation_data=(X_test,Y_test),epochs=100,batch_size=10,verbose=1)

In [ ]:
train_predict=model.predict(X_train)
test_predict=model.predict(X_test)

In [ ]:
#Scaling the predicting data back to the original range 
train_predict=scaler.inverse_transform(train_predict)
test_predict=scaler.inverse_transform(test_predict)

In [ ]:
# Getting the error in our predictions 
print(math.sqrt(mean_squared_error(Y_train,train_predict)))
print(math.sqrt(mean_squared_error(Y_test,test_predict)))

In [ ]:
### Plotting 
# Green indicates test predictions , Blue indicates original data,orange indicates train predictions.
look_back=100
trainPredictPlot = np.empty_like(data1)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[look_back:len(train_predict)+look_back, :] = train_predict
# shift test predictions for plotting
testPredictPlot = np.empty_like(data1)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(train_predict)+(look_back*2)+1:len(data1)-1, :] = test_predict
# plot the original stock graph  and  the train and test predictions 
plt.plot(scaler.inverse_transform(data1))
plt.plot(trainPredictPlot)
plt.plot(testPredictPlot)
plt.show()

In [ ]:
X_input=test_data[215:].reshape(1,-1)
X_input.shape

(1, 100)

In [ ]:
temp_input=list(X_input)
temp_input=temp_input[0].tolist()

In [ ]:
# demonstrate prediction for next 30 days

output=[] #List to store output of price over next 30 days
n_steps=100
i=0
while(i<30):
    #After the first prediction
    if(len(temp_input)>100) :
        X_input=np.array(temp_input[1:])
        print(f"{i} day input {X_input}")
        X_input=X_input.reshape(1,-1)
        X_input = X_input.reshape((1, n_steps, 1))
        yhat = model.predict(X_input, verbose=0)
        print(f"{i} day output {yhat}")
        temp_input.extend(yhat[0].tolist())
        temp_input=temp_input[1:]
        output.extend(yhat.tolist())
        i=i+1
    else:
        X_input = X_input.reshape((1, n_steps,1))
        yhat = model.predict(X_input, verbose=0)
        #First Prediction
        print(yhat[0])
        temp_input.extend(yhat[0].tolist())
        print(len(temp_input))
        output.extend(yhat.tolist())
        i=i+1
    



In [ ]:
#prediction for next 30 days in the form of a list 
output

In [ ]:
day_new=np.arange(1,101)
day_pred=np.arange(101,131)

In [ ]:
plt.plot(day_new,scaler.inverse_transform(data1[1157:]))
plt.plot(day_pred,scaler.inverse_transform(lst_output))

In [ ]:
df1=data1.tolist()
df1.extend(lst_output)
plt.plot(df1[1200:])